In [0]:
!wget 'https://datahack-prod.s3.amazonaws.com/train_file/train_jqd04QH.csv'
!wget 'https://datahack-prod.s3.amazonaws.com/test_file/test_KaymcHn.csv'

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from xgboost import XGBClassifier

In [0]:
data=pd.read_csv("/content/train_jqd04QH.csv")
data_test=pd.read_csv("/content/test_KaymcHn.csv")
data_combine=data.append(data_test)
data_combine.reset_index(inplace=True,drop=True)
data_combine.head()

In [0]:
for col,item in data_combine.iteritems():
  print(data_combine[col].value_counts(dropna=False))

In [0]:
data_combine['experience']=data_combine['experience'].replace({'>20':'21','<1':'0',np.nan:'-1'})
data_combine['last_new_job']=data_combine['last_new_job'].replace({'>4':'5','never':'0'})
data_combine['company_size']=data_combine['company_size'].replace({'10/49':'10-49','<10':'0-10'})

In [0]:
data_combine['experience']=pd.to_numeric(data_combine['experience'],errors='coerce')
data_combine['experience'].value_counts(dropna=False)

In [0]:
#Rebinning experience. It basically throws away granularity of the dataset and reduces overfittting!
bins= [-1,0,3,6,9,12,15,18,21]
labels = ['Unknown','1','2','3','4','5','6','7']
data_combine['experience'] = pd.cut(data_combine['experience'], bins=bins, labels=labels, right=False)
data_combine['experience'].value_counts()

In [0]:
data_combine['experience'].astype(str)
data_combine['experience']=data_combine['experience'].replace({'Unknown':np.nan})

In [0]:
target=data_combine['target']
numerical_data=data_combine[['training_hours','city_development_index','enrollee_id']]
categorical_data=data_combine.drop(labels=numerical_data.columns.append(pd.Index(['target'])),axis=1)

In [0]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer(missing_values=np.nan,strategy='median')
numerical_data = pd.DataFrame(my_imputer.fit_transform(numerical_data),columns=numerical_data.columns)

In [0]:
my_imputer = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
categorical_data = pd.DataFrame(my_imputer.fit_transform(categorical_data),columns=categorical_data.columns)

In [0]:
data_combine=pd.concat([categorical_data,numerical_data],axis=1)
data_combine['target']=target
data_combine.head()

In [0]:
data=data_combine[data_combine['target'].isnull()!=True]
data_test=data_combine[data_combine['target'].isnull()==True]

In [0]:
data.drop(labels='enrollee_id',axis=1,inplace=True)
data.reset_index(drop=True,inplace=True)
data.head()

In [0]:
X=data.drop(labels='target',axis=1)
y=data['target']

In [0]:
X=pd.get_dummies(X,columns=categorical_data.columns,drop_first=True)

In [0]:
train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.1,random_state=0,stratify=y)
train_X,test_X,train_y,test_y=train_X.values,test_X.values,train_y.values,test_y.values

In [0]:
from xgboost import XGBClassifier
model=XGBClassifier(silent=False, scale_pos_weight=6,learning_rate=0.0001,  
                      colsample_bytree = 0.5,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=10000, 
                      reg_alpha = 50,
                      max_depth=8, 
                      gamma=0.5,
                      n_jobs=-1,tree_method='gpu_hist',gpu_id=0)

eval_set = [(train_X, train_y), (test_X,test_y)]
eval_metric = ["auc","logloss"]
model.fit(train_X,train_y,early_stopping_rounds=1, eval_metric=eval_metric, eval_set=eval_set, verbose=False)
train_predicted=model.predict(train_X)
test_predicted=model.predict(test_X)

y_pred_prob_train = model.predict_proba(train_X)[:, 1]
y_pred_prob_test = model.predict_proba(test_X)[:, 1]

print("Training ROC AUC score",metrics.roc_auc_score(train_y,y_pred_prob_train),"\n")
print("Testing ROC AUC score",metrics.roc_auc_score(test_y,y_pred_prob_test))